# Movies Dataset Processing & Analysis

![Movies Image](plugins/assets/movies_image.png)

### Objective:
The aim of this project is to fetch historical data about movies metadata from open APIs. The gathered data will be processed, cleaned, then will be stored in the data lake using AWS S3, loaded into Snowflake, and Visualize through dashboard. 

The [Kaggle Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data) that used contains metadata for all 45,000 movies listed in the Full MovieLens Dataset. The dataset consists of movies released on or before July 2017. Data points include cast, crew, plot keywords, budget, revenue, posters, release dates, languages, production companies, countries, TMDB vote counts and vote averages.

This dataset also has files containing 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.

## Key Features of the Project:

### Data Collection:
Identify reliable open sources for movies dataset.
Use Python to fetch data using libraries such as requests, pandas, or openpyxl.


### Data Processing:
Clean and preprocess the data to ensure it's in a usable format.
Handle missing data, duplicates, and irrelevant columns.
Perform any necessary transformations (e.g., from json to dataset).

### Data Storage:
Store the data in a local database (e.g., SQLite) or a cloud-based data warehouse (e.g., Google BigQuery, AWS Redshift) for later use.


### Dashboard Development:
Use a Python visualization library (e.g., Plotly, Dash, Matplotlib) to build an interactive dashboard.
The dashboard will allow users to interact with data, filter by transport type, and visualize transport routes, schedules, or other metrics.


### Experimentation & Analysis:
Experiment with data fetching, transformation, and the integration of APIs.
Explore possible analyses such as peak-hour transport usage, performance (on-time arrivals), and comparison across routes.

### Load Libraries:

In [1]:
# Import all necessary packages
import pandas as pd
import numpy as np
import plugins.utils as utils
import snowflake.connector
from plugins.config import snow_creds, aws_creds
import json
import ast

c:\Users\iqbal\AppData\Local\Programs\Python\Python310\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


## Data Collection:

In [2]:
# Download the dataset from Kaggle
dataset_name = "rounakbanik/the-movies-dataset" 
download_folder = "./plugins/assets/data/the-movies-dataset"
utils.download_kaggle_dataset(dataset_name, download_folder)

Folder './plugins/assets/data/the-movies-dataset' has been filled.


In [3]:
# Load all of the data into DataFrames
# The column ID -> identifier will be distinguished by using m_id which stands for movie_id (Most likely to be tmdbId in links df)
credits = pd.read_csv("./plugins/assets/data/the-movies-dataset/credits.csv")
keywords = pd.read_csv("./plugins/assets/data/the-movies-dataset/keywords.csv")
links = pd.read_csv("./plugins/assets/data/the-movies-dataset/links.csv")
movies_metadata = pd.read_csv("./plugins/assets/data/the-movies-dataset/movies_metadata.csv")
ratings = pd.read_csv("./plugins/assets/data/the-movies-dataset/ratings.csv")

C:\Users\iqbal\AppData\Local\Temp\ipykernel_37328\4206996445.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv("./plugins/assets/data/the-movies-dataset/movies_metadata.csv")


## Data Processing:

In [4]:
def safe_parse_collection(x: pd.Series):
    if pd.isna(x):
        return np.nan
    
    if isinstance(x, dict):
        return x  # Already a dictionary, return as is
    
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError, TypeError):
        return np.nan
    
# GPT Generated Code
def normalize_subset(
    row: pd.Series, 
    subset_column: str, 
    id_column: str,
    prefix: str
) -> pd.DataFrame:
    """Normalize nested data from a DataFrame row.
    
    Args:
        row: Pandas Series representing a row from a DataFrame
        subset_column: Name of the column containing nested data ('cast' or 'crew')
        id_column: Name of the ID column to preserve from the original DataFrame
        
    Returns:
        Normalized DataFrame with associated credit ID
    """
    subset_data = row.get(subset_column, [])
    
    if not subset_data or not isinstance(subset_data, list):
        return pd.DataFrame()
    normalized_df = pd.json_normalize(subset_data)
    normalized_df[f"{prefix}_{id_column}"] = row[id_column]

    return normalized_df

def normalize_data(
    initial_df: pd.DataFrame, 
    subset_column: str, 
    id_column: str,
    prefix: str
) -> pd.DataFrame:
    """Normalize a DataFrame.
    
    Args:
        initial_df: DataFrame containing certain value that needs to be normalized
        subset_column: Name of the column containing nested data
        id_column: Name of the ID column to preserve from the original DataFrame
        prefix: Prefix to add to the normalized columns
    Returns:
        DataFrames with normalized data
    """

    # Normalize cast data
    new_df = initial_df.copy()
    new_df[subset_column] = new_df[subset_column].apply(safe_parse_collection)
    dfs = [
        normalize_subset(row, subset_column=subset_column, id_column=id_column, prefix=prefix)
        for _, row in new_df.iterrows()
    ]

    return  pd.concat(dfs, ignore_index=True)

#### Credits Dataframe Processing:

In [5]:
credits.sample(5)

,cast,crew,id
22784,"[{'cast_id': 3, 'character': 'Qohen Leth', 'cr...","[{'credit_id': '52fe4bd49251416c910e6f63', 'de...",157834
4408,"[{'cast_id': 2, 'character': 'Levi Rockwell', ...","[{'credit_id': '52fe4962c3a36847f81976b7', 'de...",58434
39083,"[{'cast_id': 9, 'character': '', 'credit_id': ...","[{'credit_id': '55afdccf9251411e0600c879', 'de...",350904
19674,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe4aa6c3a36847f81da483', 'de...",108901
13752,"[{'cast_id': 5, 'character': 'Jimmy Wright', '...","[{'credit_id': '52fe47a39251416c750a072d', 'de...",18869


In [6]:
c_cast = normalize_data(credits, 'cast', 'id', 'm')
c_crew = normalize_data(credits, 'crew', 'id', 'm')

In [7]:
c_cast.sample(5)

,cast_id,character,credit_id,gender,id,name,order,profile_path,m_id
390143,15,Bliss,57d2fa2dc3a3681fea00019d,0,145230,Abbes Zahmani,13,None,119193
102871,16,Briggs,556c998cc3a3685748001c68,2,166529,Philip Craig,14,/5RJvo5nkFHtRTyPeBWAedP9qa4D.jpg,15394
516279,5,Megan Harrison Hopkins,52fe46bbc3a368484e0a0d29,0,1188922,Alexa Gilmour,4,None,63809
23142,6,Dick Cutler,52fe45209251416c91028579,2,14463,David Naughton,4,/ehDE47qkUux5a18Mi7zqReBj79b.jpg,33519
360835,9,Hugh Adam David Breslin,52fe4eb9c3a36847f82a427b,2,213835,Gary Gray,7,/7skbVlsdz904DUVS8nZUInRLooU.jpg,241574


In [8]:
c_crew.sample(5)

,credit_id,department,gender,id,job,name,profile_path,m_id
340961,59b176a3c3a3681fcc032972,Crew,0,930009,Cinematography,Christina Voros,None,220493
358146,56c0a7da9251412be0000853,Sound,0,9426,Sound Re-Recording Mixer,Dean Hovey,None,299221
223726,52fe43269251416c750055b9,Production,2,63371,Producer,John McDonnell,None,10097
344795,52fe4bde9251416c910e8463,Directing,2,52909,Director,Masato Harada,/1hoOz8hAKe5U01SHn9YLzW7dVtG.jpg,158217
63770,52fe47579251416c75095e07,Editing,0,43796,Editor,Tristam Cones,None,17965


#### Keywords Dataframe Processing:

In [9]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [10]:
keywords = normalize_data(keywords, 'keywords', 'id', 'm')

In [11]:
keywords.head(5)

,id,name,m_id
0,931,jealousy,862
1,4290,toy,862
2,5202,boy,862
3,6054,friendship,862
4,9713,friends,862


#### Links Dataframe Processing:

In [12]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


#### Movies Metaddata Dataframe Processing:

In [13]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [14]:
# Inspect the first row of the movies_metadata DataFrame to decide the processing method
movies_metadata.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [27]:
def extract_dict_values(row: pd.DataFrame, column_name: str, new_column_prefix: str = None):
    new_row = safe_parse_collection(row[column_name])
    if isinstance(new_row, dict):
        for key, value in new_row.items():
            if new_column_prefix:
                row[f'{new_column_prefix}_{key}'] = value
            else:
                row[key] = value
    return row

belongs_to_collection = movies_metadata[['belongs_to_collection', 'id']].copy()
belongs_to_collection.rename(columns={'id': 'm_id'}, inplace=True)
belongs_to_collection = belongs_to_collection.apply(lambda row: extract_dict_values(row, 'belongs_to_collection'), axis=1)
movies_metadata.drop('belongs_to_collection', axis=1, inplace=True)
belongs_to_collection = belongs_to_collection.drop('belongs_to_collection', axis=1)

In [28]:
genres = normalize_data(movies_metadata, 'genres', 'id', 'm')
movies_metadata.drop(columns=['genres'], inplace=True)
genres.sample(5)

,id,name,m_id
63401,18,Drama,332835
16210,18,Drama,62327
36967,10402,Music,43939
109,36,History,11443
44986,18,Drama,71883


In [29]:
production_companies = normalize_data(movies_metadata, 'production_companies', 'id', 'm')
movies_metadata.drop(columns=['production_companies'], inplace=True)
production_companies.sample(5)

,name,id,m_id
48534,P.A.C.,1124,1871
64190,Maximage GmbH,2083,127369
52951,Rainmark Films,3346,58075
18961,Warner Bros.,6194,26654
29632,NW3 Films,69750,41163


In [30]:
movies_metadata['production_countries'].sample(5)

16302     [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
43982                                                   []
23340    [{'iso_3166_1': 'US', 'name': 'United States o...
9184     [{'iso_3166_1': 'US', 'name': 'United States o...
13315             [{'iso_3166_1': 'PL', 'name': 'Poland'}]
Name: production_countries, dtype: object

In [31]:
production_countries = normalize_data(movies_metadata, 'production_countries', 'id', 'm')
movies_metadata.drop(columns=['production_countries'], inplace=True)
production_countries.sample(5)

,iso_3166_1,name,m_id
22872,BE,Belgium,79995
24617,US,United States of America,144340
45898,DE,Germany,369594
32142,LV,Latvia,144331
33056,US,United States of America,29094


In [32]:
spoken_languages = normalize_data(movies_metadata, 'spoken_languages', 'id', 'm')
movies_metadata.drop(columns=['spoken_languages'], inplace=True)
spoken_languages.sample(5)

,iso_639_1,name,m_id
13567,ja,日本語,18289
2006,en,English,607
51582,ru,Pусский,309739
48844,en,English,419639
20684,en,English,38575


#### Ratings Dataframe Processing:

In [33]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


#### Validate whether the data is valid and related

In [49]:
# Taking sample
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [50]:
credits[credits['id'] == 110]

,cast,crew,id
302,"[{'cast_id': 9, 'character': 'Valentine Dussau...","[{'credit_id': '52fe4219c3a36847f8003dbd', 'de...",110


In [51]:
c_cast[c_cast['id'] == 110].head()

,cast_id,character,credit_id,gender,id,name,order,profile_path,m_id
2674,4,Lt. Peter 'WEAPS' Ince,52fe44ccc3a36847f80aa7e9,2,110,Viggo Mortensen,4,/gYtVNMwX96fE9F0WVkdC0SGffkn.jpg,8963
3110,14,Lucifer,535e9b58c3a36830a9005700,2,110,Viggo Mortensen,5,/gYtVNMwX96fE9F0WVkdC0SGffkn.jpg,11980
7041,11,Lalin,52fe443bc3a36847f8089ef7,2,110,Viggo Mortensen,7,/gYtVNMwX96fE9F0WVkdC0SGffkn.jpg,6075
12512,11,Roy Nord,52fe44159251416c75028531,2,110,Viggo Mortensen,2,/gYtVNMwX96fE9F0WVkdC0SGffkn.jpg,11228
23655,7,Guy Foucard,52fe44b7c3a36847f80a5f6f,2,110,Viggo Mortensen,4,/gYtVNMwX96fE9F0WVkdC0SGffkn.jpg,8744


In [52]:
c_crew[c_crew['id'] == 110].head()

,credit_id,department,gender,id,job,name,profile_path,m_id
344703,5545f2619251414c92003c16,Production,2,110,Co-Producer,Viggo Mortensen,/gYtVNMwX96fE9F0WVkdC0SGffkn.jpg,283708


In [53]:
keywords[keywords['m_id'] == 110].head()

,id,name,m_id
2028,934,judge,110
2029,1533,isolation,110
2030,2863,mannequin,110
2031,4918,shadowing,110
2032,5259,english channel,110


In [54]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0


In [55]:
links[links['tmdbId'] == 110].head()

,movieId,imdbId,tmdbId
303,306,111495,110.0


[Link validation](https://www.imdb.com/title/tt0111495/)

In [56]:
movies_metadata[movies_metadata['id'] == '110']

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
302,False,0,NaN,110,tt0111495,fr,Trois couleurs : Rouge,Red This is the third film from the trilogy by...,7.832755,/77CFEssoKesi4zvtADEpIrSKhA3.jpg,1994-05-27,0.0,99.0,Released,NaN,Three Colors: Red,False,7.8,246.0


In [58]:
belongs_to_collection[belongs_to_collection['m_id'] == '110']

,backdrop_path,id,m_id,name,poster_path
302,/AeHExfHIl70SZCea907KfEoSkfJ.jpg,131.0,110,Three Colors Collection,/rVdd23QuT5rHX7lZvuAkRRUkeCZ.jpg


In [57]:
genres[genres['m_id'] == '110']

,id,name,m_id
726,18,Drama,110
727,9648,Mystery,110
728,10749,Romance,110


In [59]:
production_companies[production_companies['m_id'] == '110']

,name,id,m_id
674,Zespół Filmowy TOR,38,110
675,Le Studio Canal+,183,110
676,France 3 Cinéma,591,110
677,Télévision Suisse-Romande,1245,110


In [60]:
production_countries[production_countries['m_id'] == '110']

,iso_3166_1,name,m_id
375,FR,France,110
376,PL,Poland,110
377,CH,Switzerland,110


In [61]:
spoken_languages[spoken_languages['m_id'] == '110']

,iso_639_1,name,m_id
414,fr,Français,110


- NOTES: With this all of the data can be concluded to be integrated and validated based on it's ID
- Additional : The data cleaning process still need to be validated